<a href="https://colab.research.google.com/github/naveedkhalid091/Learn_Agentic_AI/blob/main/step02_generative_ai_for_beginners/02(b)_RAG_implementation_with_PineconeDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RAG Implementation with PineConeDB:

**Step#1:** Install the necessary environment as follows:  

In [19]:
!pip install -qU langchain-pinecone langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 6.4 MB/s eta 0:00:00


**Step#2:** In this step we need a to initialize the Pinecone Database.

So go to the Pinecone Database, create your account and Generate API key and write your API key in colab secrets and finally get access of your vector database through the following code.   



In [20]:
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

Step#3 - Create Indexing in the Pinecone Database:

The below code can also be taken from the [PineCone Vector Database](https://www.pinecone.io/). First `Sign-in` and then go to the `Database -> Indexes` section.

You can also see the below code in the [PineCone Documentation](https://python.langchain.com/docs/integrations/vectorstores/pinecone/) by scrolling down to the Initialization Section.

In [ ]:
import time

index_name = "online-rag-project"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

pc.create_index(
        name=index_name,
        dimension=768, # Replace with your model dimensions
        metric="cosine", # Replace with your model metric
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)


In [22]:
index = pc.Index(index_name)

Now our Pinecone index is sucessfully setup, You can also varify this created Index (`online-rag-project`) into the PineCone Database and navigate to `Database -> Indexes`. After creating index, now all data will be stored in this `Index`.  


Now, we need to select the embedding model as we had studied that the data can only be saved in vector form so we need to select the embedding model (which will convert our data into vectors/numbers).

We will select the google embeddings model in our workings:

You can find the Google embedding model from the below mentioned documentation.

- [Google Embedding Documentation](https://python.langchain.com/docs/integrations/text_embedding/google_generative_ai/).


This these documentations It is clearly mentioned that you can Connect to Google's generative AI embeddings service using the `GoogleGenerativeAIEmbeddings` class, found in the `langchain-google-genai package` which we have installed at step # 1.

But keep in your mind that You also need to get access of Gemini model for accessing the embedding model.

So first access the Gemini model and then access the embeddings model as below:






In [23]:
import os

from google.colab import userdata
GOOGLE_API_KEY='GOOGLE_API_KEY'
os.environ[GOOGLE_API_KEY]=userdata.get('GOOGLE_API_KEY')



Now embed the model as below:  



**Note:** This below code is taken from [Google Embedding Documentation](https://python.langchain.com/docs/integrations/text_embedding/google_generative_ai/), from the `usage` section.

In [29]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


#### Converting text into Embeddings

In [30]:
vector = embeddings.embed_query("hello, world!")
print(vector[:5])

[0.05168594419956207, -0.030764883384108543, -0.03062233328819275, -0.02802734263241291, 0.01813093200325966]


Adding the embeddings into a wrapper (PineconeVectoreStore) so that developers can switch other databases.


In [31]:
from langchain_pinecone import PineconeVectorStore

vector_store= PineconeVectorStore(index=index, embedding=embeddings)

### Add items to vector store (Create Documents):
We can add items to our vector store by using the add_documents function and then we will reterive the added data.

In [32]:
 # 1. create Data:

from langchain_core.documents import Document

document_1=Document(
    page_content="9th class Chemistry Book",
    metadata={"source":"tweet"},
)


In [7]:
print(document_1)

page_content='9th class Chemistry Book' metadata={'source': 'tweet'}


In [33]:
# 2. Save data on vector store:

from uuid import uuid4

from langchain_core.documents import Document

document_1=Document(
    page_content="9th class Chemistry Book",
    metadata={"source":"Chem Book"},
)

document_2=Document(
    page_content="10th class Physics Book",
    metadata={"source":"physics Book"},
)

document_3=Document(
    page_content="I am feeling sick, I will not attend today's class",
    metadata={"source":"tweet"},
)


documents=[
    document_1,
    document_2,
    document_3,
]




In [34]:
len(documents)

3

In [35]:
from uuid import uuid4
uuid4()  # Generates Random id number every time


UUID('e8cf9009-a62d-457a-9cf5-1851d68f55c3')

In [36]:
## Creating IDs of all documents using the for loop method
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['09d6b130-d2dd-499c-9725-b87eeba80508',
 '5e4aebcd-f862-4d22-a04a-243d712f61f5',
 '64fce7f4-bda0-4bda-92bb-24d2d16b7092']

In [50]:
## Similarity search on the vector store:


results = vector_store.similarity_search(
    "sick",
    k=2,
    filter={"source":"tweet"}
    )

for res in results:
    print(f'''{res.page_content} [{res.metadata}]''')

I am feeling sick, I will not attend today's class [{'source': 'tweet'}]


In [54]:
# Similarity search with scores:
results = vector_store.similarity_search_with_score(
    "Who won the USA Elections?"
    )

for res, score in results:
    print(f'''[SIM={score:3f}]{res.page_content} [{res.metadata}]''')


[SIM=0.511861]10th class Physics Book [{'source': 'physics Book'}]
[SIM=0.506155]9th class Chemistry Book [{'source': 'Chem Book'}]
[SIM=0.504221]I am feeling sick, I will not attend today's class [{'source': 'tweet'}]


In [64]:
# This is dummy example, we will give the result of GAN to the LLM along with our query,
# We will learn about it later on by calling below function and importing a chatGoogleGenAI.


# copy and pasted code from
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)




In [68]:
# define a function
def answer_to_user(query:str):
  # Vector Search
  vector_results = vector_store.similarity_search(query, k=2)
  print(len(vector_results))
  # TODO: Pass to model Vector Results + User Query
  final_answer = llm.invoke (f'''Answer this user Query: {query}, Here are some ref to answer {vector_results}, ''')
  return final_answer

In [72]:
answer=answer_to_user("Sick")

2


In [73]:
answer.content

'The user is reporting feeling sick and will miss class today.  The provided documents are irrelevant to their statement; one is a tweet stating the same, and the other is a reference to a chemistry textbook.\n'

Visit the Loader url link to uplaod the documents in RAG.

(Donument Loader in RAG)[https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/]